# Graphs

> All I've got is a red guitar, three chords and the truth. The rest is up to you. --_Bob Dylan_

In [33]:
⎕IO ← 0
{}⎕SE.UCMD'box on -s=min -t=tree -f=on'
{}⎕SE.UCMD'rows on'
'span' 'stpath'⎕CY'dfns'

In [34]:
:Namespace graph
    ⎕io←0
    ⍝ https://www.dcode.fr/maze-generator
    N←{(⍸⍺)∩(⊂⍵)(+,-)(0 1)(1 0)}

    _solve←{
        (n g)←{f({{f⍳⊂⍵}¨⍵}¨m∘N¨f←⍸m←~'#'=⍵)}⍺⍺
        n[(g ##.span ⍺) ##.stpath ⍵]
    }

    showpath←{
        '⎕'@⍺⊢⍵
    }

:EndNamespace

In [4]:
maze←↑⊃⎕NGET'/Users/stefan/work/dyalog/learnapl/maze-big.txt'1

In [5]:
p ← 0 (maze graph._solve) 5004

In [1]:
⍝p graph.showpath maze

done

Rebuilding user command cache... done

## Sliding Tiles

The sliding tiles puzzle has interesting properties -- it's quite a hard graph problem to solve efficiently as the puzzle grows larger. Such solutions require a heuristic search algorithm, such as A*, and some smart heuristics. The graph is very cyclic. It's a 1kju kata on [CodeWars](https://www.codewars.com/kata/5a20eeccee1aae3cbc000090). Here's the end state for a 4×4 version:

![tiles](./IMG/slidingtiles.png)

https://gsurma.medium.com/sliding-puzzle-solving-search-problem-with-iterative-deepening-a-d7e8c14eba04

Let's start with generating the valid moves:


In [35]:
moves←{{⌽@zero ⍵⊢state}¨(,⍳⍴state)∩(0 1)(1 0)(0 ¯1)(¯1 0)+⊂zero←⊃⍸0=state←⍵}

In [36]:
⊢state←4 4⍴?⍨16 ⍝ A random puzzle state. Note: not all states have solutions!
moves state

12 6 3 4
 2 5 7 9
13 14 11 1
15 0 10 8

┌──────────┬──────────┬──────────┐
│12 6 3 4│12 6 3 4│12 6 3 4│
│ 2 5 7 9│ 2 5 7 9│ 2 5 7 9│
│13 0 11 1│13 14 11 1│13 14 11 1│
│15 14 10 8│ 0 15 10 8│15 10 0 8│
└──────────┴──────────┴──────────┘

Not too much magic there -- we find the location of the zero tile, add the valid offsets (¯1, 0, 1) in x and y, capped by the edges, and use _At_ to apply the reverse function ⌽ to swap places between the 0 tile and each of its valid 4-connected neighbors. Here's what that looks like for rank-1:

In [38]:
⌽@4 6⊢⎕←⍳8   ⍝ Swap positions 4 and 6

0 1 2 3 4 5 6 7
0 1 2 3 6 5 4 7

So the first thing to try is a breadth-first search. 